In [14]:
library(Seurat)
library(Signac)
library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(cowplot)
library(Matrix)
library(org.Hs.eg.db)
library(dplyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:ensembldb':

    filter, select


The following object is masked from 'package:AnnotationDbi':

    select


The following object is masked from 'package:Biobase':

    combine


The following objects are masked from 'package:GenomicRanges':

    intersect, setdiff, union


The following object is masked from 'package:GenomeInfoDb':

    intersect


The following objects are masked from 'package:IRanges':

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from 'package:S4Vectors':

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from 'package:BiocGenerics':

    combine, intersect, setdiff, union


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [4]:
getwd()

[1] "/home/math/hyl2016/Intergration_Benchmark/BenchmarkRealDataset/RNA_ATAC"

In [ ]:
data_sets = c("10_GSE201402_down","12_GSE178379","14_GSE156478","2_brain_ShareSeq","4_brain_ISSAAC_seq",
              "6_pbmc_granulocyte_sorted_10k","8_pbmc_unsorted_10k","11_GSE194122","13_GSE74535_scMT-seq2016",
              "1_ShareSeq_Skin","3_brain_SNARE","5_human_brain_10x","7_human_pbmc_10x","9_pbmc_unsorted_3k")

In [26]:
# batch = c('s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d10', 's3d3',
#        's3d6', 's3d7', 's4d1', 's4d8', 's4d9')

batch = c('s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d10', 's3d3',
       's3d6', 's3d7', 's4d9')
# batch = c('s4d1', 's4d8')

In [30]:
getwd()

[1] "/home/math/hyl2016/Intergration_Benchmark/BenchmarkRealDataset/RNA_ATAC/11_GSE194122"

In [29]:
setwd("11_GSE194122")

In [31]:
for (id in batch){
    path = paste0(id,"/ATAC.raw/")

    # 读取barcode.tsv文件，这将包含细胞的名称或条形码
    barcodes <- read.table(paste0(path,"barcodes.tsv"), header = FALSE, stringsAsFactors = FALSE)
    
    # 读取features.tsv文件，这将包含峰或基因的名称
    features <- read.table(paste0(path,"features.tsv"), header = FALSE, stringsAsFactors = FALSE)
    
    # 读取matrix.mtx文件，这将包含峰-细胞表达矩阵
    mat <- readMM(paste0(path,"matrix.mtx"))
    colnames(mat) <- barcodes$V1
    rownames(mat) <- features$V1
    
    # 创建Seurat对象
    pbmc <- CreateSeuratObject(counts = mat, project = "pbmc3k", min.cells = 3, min.features = 2)

    # 使用 vapply() 和 grep() 加速代码
    valid_indices <- vapply(features$V1, function(peak) {
      feature_parts <- strsplit(peak, "-")
      if (length(feature_parts[[1]]) > 0 && grepl("^chr[0-9]+$", feature_parts[[1]][1])) {
        return(TRUE)
      } else {
        return(FALSE)
      }
    }, logical(1))
    
    valid_peaks <- features$V1[valid_indices]
    
    pbmc= pbmc[valid_peaks,]
    pbmc <- RunTFIDF(pbmc)
    pbmc <- FindTopFeatures(pbmc, min.cutoff = "q0")

    var_feature = pbmc@assays$RNA@var.features
    pbmc = pbmc[var_feature,]
    path = paste0(id,"/ATAC")
    dir.create(path)
    
    writeMM(pbmc@assays$RNA@counts, paste0(path,"/matrix.mtx"))
    write.table(pbmc@assays$RNA@counts@Dimnames[1], file = paste0(path,"/features.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
    write.table(pbmc@assays$RNA@counts@Dimnames[2], file = paste0(path,"/barcodes.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
}

Performing TF-IDF normalization

Warning message in dir.create(path):
"'s1d1/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s1d2/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s1d3/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s2d1/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s2d4/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s2d5/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s3d10/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s3d3/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s3d6/ATAC' already exists"
Performing TF-IDF normalization

Warning message in dir.create(path):
"'s3d7/ATAC' already exists"
Performin

In [17]:
ing=as(pbmc@assays$RNA@counts,"TsparseMatrix")

In [24]:
writeMM(pbmc@assays$RNA@counts, paste0(path,"/matrix.mtx"), type = "coordinate", precision = "double", index = "auto")

NULL

In [23]:
pbmc@assays$RNA@counts[1,1]=0

In [4]:


# 查看Seurat对象的摘要
summary(pbmc)

Length  Class   Mode 
     1 Seurat     S4 

In [5]:
# 使用 vapply() 和 grep() 加速代码
valid_indices <- vapply(features$V1, function(peak) {
  feature_parts <- strsplit(peak, "-")
  if (length(feature_parts[[1]]) > 0 && grepl("^chr[0-9]+$", feature_parts[[1]][1])) {
    return(TRUE)
  } else {
    return(FALSE)
  }
}, logical(1))

valid_peaks <- features$V1[valid_indices]

pbmc= pbmc[valid_peaks,]
pbmc <- RunTFIDF(pbmc)
pbmc <- FindTopFeatures(pbmc, min.cutoff = "q0")

Performing TF-IDF normalization



In [6]:
var_feature = pbmc@assays$RNA@var.features
pbmc = pbmc[var_feature,]
path = "ATAC.filter"
dir.create(path)

writeMM(pbmc@assays$RNA@counts, paste0(path,"/matrix.mtx"))
write.table(pbmc@assays$RNA@counts@Dimnames[1], file = paste0(path,"/features.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
write.table(pbmc@assays$RNA@counts@Dimnames[2], file = paste0(path,"/barcodes.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

NULL

In [ ]:
# We exclude the first dimension as this is typically correlated with sequencing depth

pbmc <- RunSVD(pbmc)
pbmc <- RunUMAP(pbmc, reduction = "lsi", dims = 2:30, reduction.name = "umap.atac", reduction.key = "atacUMAP_")

In [28]:
atac = readRDS("rawdata/GSM6062731_retina_10x.atac.RDS")

In [29]:
rna = readRDS("rawdata/GSM6062732_retina_10x.rna.RDS")

In [18]:
setwd("10_GSE201402_down/")

In [30]:
getwd()

[1] "/media/Data/Program3/testcode_20230710/Masic_Intergration/BenchmarkRealDataset/RNA_ATAC/10_GSE201402_down"

In [16]:
length(intersect(atac@assays$ATAC@counts@Dimnames[[2]], rna@assays$ATAC@counts@Dimnames[[2]]))

[1] 9383

In [32]:
path = "ATAC"
dir.create(path)
writeMM(atac@assays$ATAC@counts, paste0(path,"/matrix.mtx"))
write.table(atac@assays$ATAC@counts@Dimnames[1], file = paste0(path,"/features.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
write.table(atac@assays$ATAC@counts@Dimnames[2], file = paste0(path,"/barcodes.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

Warning message in dir.create(path):
"'ATAC' already exists"


NULL

In [33]:
path = "RNA"
dir.create(path)
writeMM(rna@assays$RNA@counts, paste0(path,"/matrix.mtx"))
write.table(rna@assays$RNA@counts@Dimnames[1], file = paste0(path,"/features.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
write.table(rna@assays$RNA@counts@Dimnames[2], file = paste0(path,"/barcodes.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

Warning message in dir.create(path):
"'RNA' already exists"


NULL

In [1]:
library(Seurat)
library(Signac)
library(EnsDb.Hsapiens.v86)
library(ggplot2)
library(cowplot)
library(Matrix)

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Attaching SeuratObject

Loading required package: ensembldb

Loading required package: BiocGenerics

Warning message:
"package 'BiocGenerics' was built under R version 4.2.1"

Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep

In [2]:
setwd("/home/math/hyl2016/Intergration_Benchmark//BenchmarkRealDataset/RNA_ATAC/")

In [26]:
rna = readRDS("10_GSE201402_down/rawdata/GSM6062732_retina_10x.rna.RDS")
atac = readRDS("10_GSE201402_down/rawdata/GSM6062731_retina_10x.atac.RDS")
identical(atac@assays$ATAC@counts@Dimnames[[2]], rna@assays$RNA@counts@Dimnames[[2]])

In [30]:
features = atac@assays$ATAC@counts@Dimnames[[1]]
atac@assays$ATAC@counts@Dimnames[[1]] = gsub(":", "-", features)

In [33]:
pbmc <- CreateSeuratObject(counts =atac@assays$ATAC@counts , project = "pbmc3k", min.cells = 3, min.features = 2)
features = pbmc@assays$RNA@counts@Dimnames[[1]]
# modified_string <- gsub(":", "-", features)
# 使用 vapply() 和 grep() 加速代码
valid_indices <- vapply(features, function(peak) {
  feature_parts <- strsplit(peak, "-")
  if (length(feature_parts[[1]]) > 0 && grepl("^chr[0-9]+$", feature_parts[[1]][1])) {
    return(TRUE)
  } else {
    return(FALSE)
  }
}, logical(1))

valid_peaks <- features[valid_indices]
pbmc= pbmc[valid_peaks,]
pbmc <- RunTFIDF(pbmc)
pbmc <- FindTopFeatures(pbmc, min.cutoff = "q0")

var_feature = pbmc@assays$RNA@var.features
pbmc = pbmc[var_feature,]
path = paste0("./10_GSE201402_down","/ATAC")
dir.create(path)

writeMM(pbmc@assays$RNA@counts, paste0(path,"/matrix.mtx"))
write.table(pbmc@assays$RNA@counts@Dimnames[1], file = paste0(path,"/features.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
write.table(pbmc@assays$RNA@counts@Dimnames[2], file = paste0(path,"/barcodes.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

Performing TF-IDF normalization

Warning message in dir.create(path):
"'./10_GSE201402_down/ATAC' already exists"


NULL

In [37]:
write.csv(rna@meta.data,"./10_GSE201402_down/metadata.csv")

In [38]:

path = paste0("./10_GSE201402_down","/RNA")
dir.create(path)

writeMM(rna@assays$RNA@counts, paste0(path,"/matrix.mtx"))
write.table(rna@assays$RNA@counts@Dimnames[1], file = paste0(path,"/features.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
write.table(rna@assays$RNA@counts@Dimnames[2], file = paste0(path,"/barcodes.tsv"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

Warning message in dir.create(path):
"'./10_GSE201402_down/RNA' already exists"


NULL